In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)


In [141]:
hr_data = pd.read_csv('hr_analytics_train.csv')
df = hr_data.copy()

In [142]:
df.set_index('employee_id', inplace=True)

In [143]:
df.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [144]:
# fill education missing values with bachelors'
df['education'].value_counts()
df['education'].fillna("Bachelor's", inplace = True)
df.education.isnull().sum()

0

In [145]:
# Filling missing values: we have filled previous_yr_rating missing values with 3.9 if is_promoted = 1 else 3.26

df['previous_year_rating'] = df.apply(
    lambda row: 3.267910689246036 if np.isnan(row['previous_year_rating']) and row['is_promoted']==0 
    else (3.986139986139986 if np.isnan(row['previous_year_rating']) and row['is_promoted']==1 else row['previous_year_rating'])
    , axis=1)

In [146]:
df.previous_year_rating.isnull().sum()

0

In [147]:
print(np.mean(df.previous_year_rating))
print(df.previous_year_rating.max())
print(df.previous_year_rating.max())
print(df.previous_year_rating.min())

3.329082313058271
5.0
5.0
1.0


In [148]:
df.columns

Index(['department', 'region', 'education', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service',
       'KPIs_met >80%', 'awards_won?', 'avg_training_score', 'is_promoted'],
      dtype='object')

In [149]:
df = df.rename(columns = {"KPIs_met >80%":"KPIs_met_80", "awards_won?":"awards_won"}) 

In [150]:
#emp_id = df['employee_id']
#df = df.drop(columns=['employee_id'])
df = pd.get_dummies(df, drop_first=True)
df.head()

,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met_80,awards_won,avg_training_score,is_promoted,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,region_region_10,region_region_11,region_region_12,region_region_13,region_region_14,region_region_15,region_region_16,region_region_17,region_region_18,region_region_19,region_region_2,region_region_20,region_region_21,region_region_22,region_region_23,region_region_24,region_region_25,region_region_26,region_region_27,region_region_28,region_region_29,region_region_3,region_region_30,region_region_31,region_region_32,region_region_33,region_region_34,region_region_4,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,education_Below Secondary,education_Master's & above,gender_m,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
65438,1,35,5.0,8,1,0,49,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
65141,1,30,5.0,4,0,0,60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7513,1,34,3.0,7,0,0,50,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2542,2,39,1.0,10,0,0,50,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
48945,1,45,3.0,2,0,0,73,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [151]:
df['kpi_award'] = df['KPIs_met_80'] + df['awards_won']

In [129]:
#df = df.drop(columns=['no_of_trainings','age'])

In [130]:
'''df2 = df.copy()
df2 = df2.drop(columns=['is_promoted'])
df2.corrwith(df.is_promoted).plot.bar(figsize=(20,10), fontsize=15, grid=True, title="correlation with is_promoted")'''

'df2 = df.copy()\ndf2 = df2.drop(columns=[\'is_promoted\'])\ndf2.corrwith(df.is_promoted).plot.bar(figsize=(20,10), fontsize=15, grid=True, title="correlation with is_promoted")'

In [152]:
# lets now check with dropping several cols how that impacts the overall models
X = df.drop(columns=['is_promoted'])
#X = df.drop(columns=['is_promoted'])

In [153]:
Y = df['is_promoted']

In [133]:

# Polynomial features
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
combinations = list(combinations(list(X.columns), 2))
col_names = list(X.columns) + ['_'.join(x) for x in combinations ]
print(col_names)
print(len(col_names))
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias = False)
X = poly.fit_transform(X)
#type(df)
X = pd.DataFrame(X)
type(X)
X.columns = col_names

indices = []
for i,x in enumerate(list((X==0).all())):
    if x == True:
        indices.append(i)
X = X.drop(X.columns[indices], axis =1)        


['previous_year_rating', 'length_of_service', 'KPIs_met_80', 'awards_won', 'avg_training_score', 'department_Finance', 'department_HR', 'department_Legal', 'department_Operations', 'department_Procurement', 'department_R&D', 'department_Sales & Marketing', 'department_Technology', 'region_region_10', 'region_region_11', 'region_region_12', 'region_region_13', 'region_region_14', 'region_region_15', 'region_region_16', 'region_region_17', 'region_region_18', 'region_region_19', 'region_region_2', 'region_region_20', 'region_region_21', 'region_region_22', 'region_region_23', 'region_region_24', 'region_region_25', 'region_region_26', 'region_region_27', 'region_region_28', 'region_region_29', 'region_region_3', 'region_region_30', 'region_region_31', 'region_region_32', 'region_region_33', 'region_region_34', 'region_region_4', 'region_region_5', 'region_region_6', 'region_region_7', 'region_region_8', 'region_region_9', 'education_Below Secondary', "education_Master's & above", 'gender

In [154]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=0)

In [155]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(43846, 54)
(10962, 54)
(43846,)
(10962,)


In [156]:
# feature scaling
from sklearn.preprocessing import StandardScaler

In [157]:
sc_X = StandardScaler()
X_train2 = pd.DataFrame(sc_X.fit_transform(X_train))
X_test2 = pd.DataFrame(sc_X.fit_transform(X_test))

In [158]:
X_train2.columns = X_train.columns.values
X_test2.columns = X_test.columns.values

X_train2.index = X_train.index.values
X_test2.index = X_test.index.values

In [159]:
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier(base_score=0.5,n_estimators=250, max_depth=6,learning_rate= 0.25, objective= 'binary:logistic')
xgb_classifier.fit(X_train,y_train)
y_pred = xgb_classifier.predict(X_test)

In [160]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(precision_score(y_test, y_pred))

0.945356686735997
0.44082519001085774
0.5754783841247343
0.8285714285714286


In [24]:
len(X.columns)
X.columns

Index(['no_of_trainings', 'age', 'previous_year_rating', 'length_of_service',
       'KPIs_met >80%', 'awards_won?', 'avg_training_score',
       'department_Finance', 'department_HR', 'department_Legal',
       ...
       'department_Technology_recruitment_channel_referred',
       'department_Technology_recruitment_channel_sourcing',
       'education_Below Secondary_gender_m',
       'education_Below Secondary_recruitment_channel_referred',
       'education_Below Secondary_recruitment_channel_sourcing',
       'education_Master's & above_gender_m',
       'education_Master's & above_recruitment_channel_referred',
       'education_Master's & above_recruitment_channel_sourcing',
       'gender_m_recruitment_channel_referred',
       'gender_m_recruitment_channel_sourcing'],
      dtype='object', length=178)

In [25]:
from xgboost import DMatrix
len(xgb_classifier.feature_importances_)
xgb_classifier.feature_importances_

array([0.        , 0.00361933, 0.02106125, 0.00579898, 0.00750243,
       0.03414347, 0.02062326, 0.        , 0.01972297, 0.00110747,
       0.02855055, 0.02685565, 0.00498056, 0.00971849, 0.00667909,
       0.00101799, 0.00245111, 0.0017509 , 0.        , 0.00250651,
       0.00357131, 0.00400464, 0.00346905, 0.00234366, 0.        ,
       0.00340117, 0.00659561, 0.        , 0.        , 0.00333827,
       0.00289887, 0.00147984, 0.00496009, 0.00219613, 0.        ,
       0.00322465, 0.00565467, 0.00195519, 0.00218193, 0.00314471,
       0.00334138, 0.01407051, 0.0027468 , 0.00333267, 0.004199  ,
       0.00331395, 0.0051147 , 0.0028105 , 0.00362775, 0.00349294,
       0.00324993, 0.0031238 , 0.0022462 , 0.0034584 , 0.00337957,
       0.00208428, 0.00346187, 0.01497702, 0.08715983, 0.02740796,
       0.0109747 , 0.00129882, 0.        , 0.        , 0.00604909,
       0.00506174, 0.00192453, 0.01101343, 0.00964408, 0.00162162,
       0.00307194, 0.00305713, 0.00267681, 0.00384272, 0.00377

In [26]:
from sklearn.metrics import roc_auc_score

print("roc_auc test set", roc_auc_score(y_test, xgb_classifier.predict_proba(X_test)[:,1]))

print("roc_auc training set", roc_auc_score(y_train, xgb_classifier.predict_proba(X_train)[:,1]))


roc_auc test set 0.9101770688061683
roc_auc training set 0.9868944059210061


In [27]:
# K -fold cross validation
from sklearn.model_selection import cross_val_score

f1_score = cross_val_score(estimator= xgb_classifier, X = X_train, y= y_train, cv = 10, scoring='f1')
recall = cross_val_score(estimator= xgb_classifier, X = X_train, y= y_train, cv = 10, scoring='recall')

In [28]:
print('f1-score: ',np.mean(f1_score))
print('recall: ',np.mean(recall))

f1-score:  0.5582185868013008
recall:  0.4166053475935828


In [138]:
xgb_classifier.feature_importances_

TypeError: 'numpy.ndarray' object is not callable

In [25]:
# Grid Search
from sklearn.model_selection import GridSearchCV

params = {'base_score':[0.5,0.6,0.4], 'learning_rate':[0.15,0.2,0.25],
         'max_depth':[5,6,7], 'n_estimators': [200,250,300]}

gridSearchCV = GridSearchCV(estimator=xgb_classifier,param_grid=params,cv=5,scoring='f1',n_jobs=-1)
gridSearchCV.fit(X_train,y_train)


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=4, min_child_weight=1,
                                     missing=None, n_estimators=150, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'base_score': [0.5, 0.6, 0.4],
                         'learning_rate': [0.05, 0.1, 0.15],
                         'max_depth': [5, 6, 7],
             

In [26]:
gridSearchCV.best_score_

0.5599799921177628

In [27]:
gridSearchCV.best_params_

{'base_score': 0.6, 'learning_rate': 0.15, 'max_depth': 6, 'n_estimators': 200}